#  深層学習のチューニング

- **[2.1 ハイパーパラメータ](#2.1-ハイパーパラメータ)**
    - **[2.1.1 ハイパーパラメータ](#2.1.1-ハイパーパラメータ)**
<br><br>
- **[2.2 ネットワーク構造](#2.2-ネットワーク構造)**
    - **[2.2.1 ネットワーク構造](#2.2.1-ネットワーク構造)**
<br><br>
- **[2.3 ドロップアウト](#2.3-ドロップアウト)**
    - **[2.3.1 ドロップアウト](#2.3.1-ドロップアウト)**
<br><br>
- **[2.4 活性化関数](#2.4-活性化関数)**
    - **[2.4.1 活性化関数](#2.4.1-活性化関数)**
    - **[2.4.2 シグモイド関数](#2.4.2-シグモイド関数)**
    - **[2.4.3 ReLU](#2.4.3-ReLU)**
<br><br>
- **[2.5 損失関数](#2.5-損失関数)**
    - **[2.5.1 損失関数](#2.5.1-損失関数)**
    - **[2.5.2 二乗誤差](#2.5.2-二乗誤差)**
    - **[2.5.3 クロスエントロピー誤差](#2.5.3-クロスエントロピー誤差)**
<br><br>
- **[2.6 最適化関数](#2.6-最適化関数)**
    - **[2.6.1 最適化関数](#2.6.1-最適化関数)**
<br><br>
- **[2.7 学習率](#2.7-学習率)**
    - **[2.7.1 学習率](#2.7.1-学習率)**
<br><br>
- **[2.8 ミニバッチ学習](#2.8-ミニバッチ学習)**
    - **[2.8.1 ミニバッチ学習](#2.8.1-ミニバッチ学習)**
<br><br>
- **[2.9 反復学習](#2.9-反復学習)**
    - **[2.9.1 反復学習](#2.9.1-反復学習)**
<br><br>
- **[2.10 添削問題](#2.10-添削問題)**

***

## 2.1 ハイパーパラメータ

### 2.1.1 ハイパーパラメータ

深層学習手法を使うと、分類あるいは回帰のアルゴリズムをほとんど自動的に生成できるためとても便利です。  
また、ニューラルネットワークモデルはいろいろな場面に適用させることができ汎用的です。  

しかし、ネットワークを構成する際に人が調整するべきパラメーターがいくつか存在します。  
これらは **<font color=#AA0000>ハイパーパラメータ</font>** と呼ばれます。  

以下は、chapter1のMNIST分類のコードに少しだけ変更を加え、またいくつかのパラメーターを明示した典型的な深層学習手法のコードです。  
以下のコードのどこがハイパーパラメータに相当するのかを見ていきます。

```python
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
# ハイパーパラメータ：活性化関数
model.add(Activation("sigmoid"))
# ハイパーパラメータ：隠れ層の数、隠れ層のチャンネル数
model.add(Dense(128))
model.add(Activation("sigmoid"))
# ハイパーパラメータ：ドロップアウトする割合（rate）
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

# ハイパーパラメータ：学習率（Ir）
sgd = optimizers.SGD(lr=0.01)

# ハイパーパラメータ：最適化関数（optimizer）
# ハイパーパラメータ：誤差関数（loss）
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

# ハイパーパラメータ：バッチサイズ（batch_size）
# ハイパーパラメータ：エポック数（epochs）
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))
```
（metricsは評価関数なので、学習自体には関係ありません。評価関数については機械学習概論を参照してください。）

上記のようにハイパーパラメータはたくさんあります。  
ハイパーパラメータは自動で変更できないものです。これらを適切に設定しないと正しく学習が行われません。  
自分で新しくモデルを作る時には最適なハイパーパラメータを吟味する必要があります。    
このchapterではそれぞれのハイパーパラメータの意味を理解し、自分でネットワークを構成、調整ができるようにしていきます。

#### 問題

- ハイパーパラメータについて説明した文として正しいものを選んでください。

- ハイパーパラメータは学習時にモデルが自動的に調整する。
- ハイパーパラメータは自分で調整する必要がある。
- ハイパーパラメータは適切に設定するのが良いが、適切でなくとも多くの場合問題なく学習は進行する。

- パラメーターのうち人が調整するパラメーターをハイパーパラメーターと言います。

#### 解答

- ハイパーパラメータは自分で調整する必要がある。

***

## 2.2 ネットワーク構造

### 2.2.1 ネットワーク構造 

**ネットワークの構造（隠れ層の数、隠れ層のユニット数）は自由に決めて生成することができます。**  

一般に、隠れ層やユニット数を多くすると、多彩な関数が表現できるようになります。  
しかし、隠れ層が多くなると、入力層に近い重みを適切に更新するのが難しく学習がなかなか進みにくくなったり、  
隠れ層のユニット数が多くなると重要性の低い特徴量を抽出してしまい過学習(汎化性能が低くなった状態)をしやすくなるなど、  
適切にネットワークの構造を設定する必要があります。

ネットワーク構造は理論で裏付けて定めることが難しく、実際には他の似たような実装例を参考にするなど経験に基づいて決定される傾向があります。

#### 問題

- 次の3つの中から一番精度の出るモデルを予想し、以下のコードを一部変更してください。
- ネットワーク構造、特に隠れ層の構造がモデルの学習に与える影響を確認してください
 - A: ユニット数256の全結合隠れ層1つと、ユニット数128の全結合隠れ層1つをもつモデル（ハイパーパラメーターの章のものと同じモデル）
 - B: ユニット数256の全結合隠れ層1つと、ユニット数128の全結合隠れ層3つをもつモデル
 - C: ユニット数256の全結合隠れ層1つと、ユニット数1568の全結合隠れ層1つをもつモデル
- 条件は以下のようにします。
 - コードを二行コメントアウトし、他は変えないでください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))

def funcA():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcB():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcC():
    model.add(Dense(1568))
    model.add(Activation("sigmoid"))

# 二つコメントアウトしてください。
#---------------------------
funcA()
funcB()
funcC()
#---------------------------

model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

- 全パターン試してみてください。

#### 解答

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))

def funcA():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcB():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcC():
    model.add(Dense(1568))
    model.add(Activation("sigmoid"))

# 二つコメントアウトしてください。
#---------------------------
funcA()
#funcB()
#funcC()
#---------------------------

model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.3 ドロップアウト

### 2.3.1 ドロップアウト

**<font color=#AA0000>ドロップアウト</font>** は、 **過学習を防ぎモデルの精度をあげるための手法の一つ** です。  

ドロップアウトを使うと、ユニットの一部が学習のたびにランダムに削除（より正確には0で上書き）されます。  
これにより、ニューラルネットは特定のニューロンの存在に依存できなくなり、より **汎用的な（学習データ以外でも通用しやすい）特徴を学習する** ようになります。 
その結果、学習データに対する過学習を防ぐことができます。  

ドロップアウトは以下のようにして使います。
```python
model.add(Dropout(rate=0.5))
```
ここでrateは削除するユニットの割合です。

**ドロップアウトを使う位置、引数のrateはともにハイパーパラメータです。**

#### 問題

- ドロップアウトを実装して、訓練データとテストデータそれぞれの正解率が近くなっていることを確認してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))

#---------------------------
#        ここを書いて下さい       
#---------------------------

model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

- ドロップアウトの実装は、`Dropout()`を用います。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
#---------------------------
model.add(Dropout(rate=0.5))
#---------------------------
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

***

## 2.4 活性化関数

### 2.4.1 活性化関数

**<font color=#AA0000>活性化関数</font>** とは、主に全結合層の後に適用する関数で、もともとニューロンの発火に相当していたものです。  

全結合層では、入力を線形変換したものを出力しますが、 **活性化関数を用いることで非線形性をもたせます** 。

活性化関数を使わない場合、以下のような一本の直線で分離できない（線形分離不可能）データは分類できないことが数学的にわかっています。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_10.png">

**非線形性をもたせることで、適切に学習が進めば線形分離不可能なモデルでも必ず分類することができます。**


**活性化関数もハイパーパラメータです。**  
よく使われる活性化関数はいくつかあり、適切に選ぶ必要があります。  

#### 問題

- 活性化関数を使う理由として正しい選択肢を選んでください。

- モデルに線形性をもたせ、線形分離可能なデータに対応させるため。
- モデルに線形性をもたせ、線形分離不可能なデータに対応させるため。
- モデルに非線形性をもたせ、線形分離可能なデータに対応させるため。
- モデルに非線形性をもたせ、線形分離不可能なデータに対応させるため。

- モデルが線型性の場合は、線形分離不可能なデータを分類できません。

#### 解答

- モデルに非線形性をもたせ、線形分離不可能なデータに対応させるため。

***

### 2.4.2 シグモイド関数

活性化関数として用いられる関数の1つに **<font color=#AA0000>シグモイド関数</font>** というものがあり、この関数は次式で与えられます。

$$
sigmoid(x) = \frac{1}{1+e^{-x}}
$$

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_20.png">

青いグラフがシグモイド関数で、オレンジ色のグラフがシグモイド関数の導関数です。

#### 問題

- 説明文のグラフからわかる、シグモイド関数の説明として正しいものを1つ選んでください。

- 出力は必ず区間 `(0,1)` に収まるので、極端な出力値が少ない。
- どのような区間にも収まらず、極端な出力値が生成される可能性がある。 
- 出力が広い値をとるので、学習速度が早くなる。
- 出力の範囲が限られていないので、学習速度が遅くなる。

- 縦軸の値に注目しましょう。

#### 解答

出力は必ず区間 `(0,1)` に収まるので、極端な出力値が少ない。

***

### 2.4.3 ReLU

もうひとつ活性化関数によく用いられる **<font color=#AA0000>ReLU（ランプ関数）</font>** というものについて説明します。  
ReLUはRectified Linear Unitの略で次式のような関数です。

$$
\mathrm{ReLU}(x) = \begin{cases} 0 (x<0) \\ x (x\geq 0)\end{cases}
$$

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_30.png">

青いグラフがReLUで、オレンジ色のグラフがReLUの導関数です。

#### 問題

- 説明文のグラフからわかる、ReLUの説明として正しいものを1つ選んでください。

- 出力は必ず区間(0,1)に収まるので、極端な出力値が少ない。
- 出力はどのような区間にも収まらず、極端な出力値が生成されうる。
- 出力が広い値をとるので、学習速度が遅くなる。
- 出力の範囲が限られているので、学習速度が遅くなる。

- 一般的に、出力が大きい値だと学習速度は早くなります。

#### 解答

- 出力はどのような区間にも収まらず、極端な出力値が生成されうる。

***

## 2.5 損失関数

### 2.5.1 損失関数

学習時に、モデルの出力と教師データとの差（間違え具合）を評価する関数を **<font color=#AA0000>損失関数（誤差関数）</font>** といいます。

損失関数には **二乗誤差** や **クロスエントロピー誤差** などが用いられます。  

この損失関数を最小化するように誤差逆伝播法という手法で各層の重みは更新されます。  

損失関数としてなぜ正解率を使わないのか疑問に思うかもしれません。  
確かに正解率を使うことはできなくはないですが、  
モデルがどの程度間違えているかやどういう間違え方（どのクラスだと誤認したのか）をしているかなどを総合的に評価するために、二乗誤差やクロスエントロピー誤差を損失関数として使うのが一般的です。

#### 問題

 - 損失関数の性質を示した文として適切なものを選んでください。

- 一般に、損失関数を最大化するように各層の重みを更新する。
- 損失関数は重みを更新する際に重要な役割を持つので、適切なものを選ぶ必要がある。
- 損失関数には正解率を求める式をそのまま使うのが良い。
- 損失関数は1種類しかないので、これはハイパーパラメータではない。

- 損失関数には様々な種類が存在し、値を最小化するように重みの更新をします。

#### 解答例

- 損失関数は重みを更新する際に重要な役割を持つので、適切なものを選ぶ必要がある。

***

### 2.5.2 二乗誤差

**<font color=#AA0000>二乗誤差</font>** は、最小二乗法として統計学など様々な分野で用いられる誤差関数です。

$$
{E} = \sum_{i=1}^N ({t_i - y_i})^{2}
$$

**連続値の評価に優れているため主に回帰モデルの誤差関数** として使われます。上式の$y_i$、$t_i$はそれぞれ、予測ラベル、正解ラベルを表しています。

#### 問題

- 二乗誤差の説明として正しいものを1つ選んでください。


- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい
- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しにくい
- 分類に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい
- 分類に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しにくい

- 下に凸の放物線をイメージしてください。

#### 解答

- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい

***

### 2.5.3 クロスエントロピー誤差

**<font color=#AA0000>クロスエントロピー誤差</font>** は、 **二値分類の評価に特化しているため、主に分類モデルの誤差関数** として使われます。  

$$
E=\sum_{i=1}^N (-t_i\log y_i-(1-t_i)\log (1-y_i))
$$

それでは、この関数がどのような特性をもつのかみていきましょう。

(i) $t_i$ << $y_i$ のとき
$-t_i\log y_i$ はほぼ0で、 $-(1-t_i)\log (1-y_i)$ は正の無限大です。  
(ii) $t_i$ >> $y_i$ のとき
$-t_i\log y_i$ は正の無限大で、 $-(1-t_i)\log (1-y_i)$ はほぼ0です。  
(iii) $t_i$ ≒ $y_i$ のとき  $-t_i\log y_i-(1-t_i)\log (1-y_i)$ は 0.69... ~ 0 の値を取ることが簡単な計算で求まります。

したがって $-t_i\log y_i-(1-t_i)\log (1-y_i)$ は、  
$|t_i - y_i|$ が大きいとき極端に大きな値を返し、$|t_i - y_i|$ が小さいとき0に近い値をとることがわかります。

分類の学習において、予測ラベル$y_i$と正解ラベル$t_i$の値は近いほど良いのでこの関数は有用です。
これらのことから、クロスエントロピー誤差は、 **0~1の2つの数の差を評価する上で合理的な関数** であると言えます。

#### 問題

- クロスエントロピー誤差について正しい選択肢を選んでください。


- 正解ラベルと予測ラベルの値が近いほど小さい値をとなる。
- 多クラス分類に特化した誤差関数である。
- 回帰問題に頻繁に用いられる誤差関数である。

- クロスエントロピーは二値分類の評価に特化しており、主に分類モデルの誤差関数として使われます。

#### 解答

- 正解ラベルと予測ラベルの値が近いほど小さい値をとなる。

***

## 2.6 最適化関数

### 2.6.1 最適化関数

重みの更新は、誤差関数を各重みで微分した値を元に、更新すべき方向とどの程度更新するかを決めます。  
微分によって求めた値を、 **学習率、エポック数、過去の重みの更新量など** を踏まえてどのように重みの更新に反映するかを定めるのが **<font color=#AA0000>最適化関数</font>** です。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_40.gif">

最適化関数はハイパーパラメータです。
上図が示すように、最適化関数にはいくつか種類があり、正しく選択しないと学習に悪影響を|及ぼします。

> 出典:http://cs231n.github.io/neural-networks-3/#add

#### 問題

 - 最適化関数の性質を示した文として適切なものを選んでください。

- 一般に、最適化関数を最大化するように各層の重みを更新する。
- 最適化関数はどれを選んでも最適化されるため、選ぶ必要はない。
- 最適化関数は損失関数、エポック数など複数の情報を踏まえて重みの更新を行う。
- 最適化関数には1種類しかないので、これはハイパーパラメータではない。

- 最適化関数は様々な要素を踏まえて重みの更新を行いますが、手法によって重みの更新の仕方が異なり、ハイパーパラメータの一種となっています。

#### 解答

- 最適化関数は損失関数、エポック数など複数の情報を踏まえて重みの更新を行う。

***

## 2.7 学習率

### 2.7.1 学習率

**<font color=#AA0000>学習率</font>** とは、 **各層の重みを一度にどの程度変更するかを決めるハイパーパラメーター** です。

以下は、最小化を行おうとしているモデルと、学習率が与える影響を図示したものです。右上の点が初期値です。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_50.png">

1. 学習率が低すぎて、ほどんど更新が進んでいません。
1. 適切な学習率のため、少ない回数で値が収束しています。
1. 収束はしますが、値が大きいため、更新の仕方に無駄があります。
1. 学習率が大きすぎて、値が発散してしまっています。（上側に更新されており、値がどんどん大きくなっています。） 

このように、 **損失関数に対して適切な学習率を設定する必要** があります。

#### 問題

- 次の3つの中から一番精度の出る学習率を予想し、以下のコードの一部を変更してください。
- 学習率がモデルの学習に与える影響を確認してください。
  - `funcA() lr: 0.01`
  - `funcB() lr: 0.1`
  - `funcC() lr: 1.0`
- コードを二行コメントアウトし、他は変えないでください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))


def funcA():
    global lr
    lr = 0.01

def funcB():
    global lr
    lr = 0.1

def funcC():
    global lr
    lr = 1.0

# 二つコメントアウトして学習率を決めてください。
#---------------------------
funcA()
funcB()
funcC()
#---------------------------

sgd = optimizers.SGD(lr=lr)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

- 全てのパターンを試してみましょう。

#### 解答

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))


def funcA():
    global lr
    lr = 0.01

def funcB():
    global lr
    lr = 0.1

def funcC():
    global lr
    lr = 1.0

# 二つコメントアウトして学習率を決めてください。
#---------------------------
#funcA()
funcB()
#funcC()
#---------------------------

sgd = optimizers.SGD(lr=lr)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.8 ミニバッチ学習

### 2.8.1 ミニバッチ学習

モデルの学習を行う際、一度にモデルに渡す入力データの数は変えることができます。  
一度に渡すデータの数を、 **バッチサイズ** といい、これもハイパーパラメータです。

一度に複数のデータを渡した時、モデルはそれぞれのデータでの損失と損失関数の勾配（重みをどのように更新するべきか）を求めますが、  
重みの更新は、１回のみ、求めた勾配の平均を使って行われます。

**複数のデータを用いて重みの更新を行うことで、極端に変わったデータの影響をあまり受けずに済み、また並列計算が行えるので計算時間を短縮することができます。**  
一方、複数のデータを用いて重みの更新を行うと、極端な重みの更新が発生しなくなり、損失関数の局所解から抜け出せなくなる恐れがあります。

**癖の強いデータが多い時はバッチサイズを大きくする、同じようなデータが多いときはバッチサイズを小さくする** などと、
バッチサイズをうまく調整する必要があります。  

バッチサイズを1とする手法を**オンライン学習(確率的勾配法)**  

バッチサイズを全データ数とする手法を**バッチ学習（最急降下法）**

これらの中間となる手法を**ミニバッチ学習**と言います。

#### 問題

- 次の3つの中から一番精度の出るバッチサイズを予想し、以下のコードの一部を変更してください。
- バッチサイズがモデルの学習に与える影響を確認してください。
  - `funcA() batch_size: 16`
  - `funcB() batch_size: 32`
  - `funcC() batch_size: 64`
- コードを二行コメントアウトし、他は変えないでください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global batch_size
    batch_size = 16

def funcB():
    global batch_size
    batch_size = 32

def funcC():
    global batch_size
    batch_size = 64

# 二つコメントアウトしてbatch_sizeを決めてください。
#---------------------------
# batch_size: 16
funcA()
# batch_size: 32
funcB()
#batch_size: 64
funcC()
#---------------------------

model.fit(X_train, y_train, batch_size=batch_size, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

- 全てのパターン実行してみましょう。

#### 解答

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global batch_size
    batch_size = 16

def funcB():
    global batch_size
    batch_size = 32

def funcC():
    global batch_size
    batch_size = 64

# 二つコメントアウトしてbatch_sizeを決めてください。
#---------------------------
#funcA()
#funcB()
funcC()
#---------------------------

model.fit(X_train, y_train, batch_size=batch_size, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.9 反復学習

### 2.9.1 反復学習

一般に、モデルの精度をあげるため同じ訓練データを使って何度か学習させるということを行います。これを **反復学習** といいます。  

この学習を行う回数を **エポック数** といい、これもハイパーパラメータです。

エポック数は大きくすればモデルの精度が上がり続ける、というものではありません。  
正解率は途中から伸びなくなるだけでなく、繰り返し学習をすることで損失関数を最小化させようとして過学習が起こります。  
適切なタイミングで学習を打ち切ることが必要となってきます。

#### 問題

- 次の3つの中から一番精度の出るエポック数を予想し、以下のコードの一部を変更してください。
- エポック数がモデルの学習に与える影響を確認してください。
  - `funcA() epochs: 5`
  - `funcB() epochs: 10`
  - `funcC() epochs: 60`
- コードを二行コメントアウトし、他は変えないでください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:1500]
X_test = X_test.reshape(X_test.shape[0], 784)[:6000]
y_train = to_categorical(y_train)[:1500]
y_test = to_categorical(y_test)[:6000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
# 今回はDropoutを使いません。
#model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global epochs
    epochs = 5

def funcB():
    global epochs
    epochs = 10

def funcC():
    global epochs
    epochs = 60

# 二つコメントアウトしてエポック数を決めてください。
#---------------------------
# epochs: 5
funcA()
# epochs: 10
funcB()
# epochs: 60
funcC()
#---------------------------

history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

- 全てのパターン実行してみましょう。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:1500]
X_test = X_test.reshape(X_test.shape[0], 784)[:6000]
y_train = to_categorical(y_train)[:1500]
y_test = to_categorical(y_test)[:6000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
# 今回はDropoutを使いません。
#model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global epochs
    epochs = 5

def funcB():
    global epochs
    epochs = 10

def funcC():
    global epochs
    epochs = 60

# 二つコメントアウトしてエポック数を決めてください。
#---------------------------
#funcA()
funcB()
#funcC()
#---------------------------

history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

### ディープラーニングのまとめ

- このコースのまとめ
- 次にオススメのコース

#### 問題

- 次の動画をみてください。

In [ ]:
https://www.youtube.com/embed/YWCbzfqt2GU

***

## 2.10 添削問題

chapter2でハイパーパラメータを一通り学習し、自分で適切なディープニューラルネットワークを構成、調整できるようになりました。  
ここではハイパーパラメータのチューニングのみでMNIST分類の精度向上を行い、ハイパーパラメータをより深く理解していきます。

#### 問題

- MNISTの分類モデルをディープニューラルネットワークで実装してください。条件は以下のようにします。
 - テストデータによる正解率は87%以上を出してください。
 - テストデータ、訓練データそれぞれによる正解率の差は1%未満になるようにしてください。
 - エポック数は5で固定とします。
 - X_train, y_train, X_test, y_test の定義文は変更しないでください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:10000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:10000]

#---------------------------
#        ここを書いて下さい       
#---------------------------

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 2.1.1のコードを参考にして構いません。コード中のハイパーパラメータを一つ変更するだけでも正解率87%を出すことができます。
- 重みの初期化の仕様上正解率は毎回同じ値にはなりませんが、ほぼ毎回87%以上出る状態を目指してください。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:10000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:10000]

#---------------------------
model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1)
#---------------------------

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***